# Sudoku de Consola en Python

Este cuaderno describe paso a paso como se creo un juego de Sudoku jugable en la consola, usando colores ANSI para resaltar los diferentes elementos del tablero.

---

## 1. Definición de códigos de color

Primero, definimos una clase con las secuencias de escape ANSI para colorear la salida en consola. Usaremos:

- Verde para los números originales del tablero.
- Azul para los números que ingresa el usuario.
- Amarillo para las pistas.
- Gris para las casillas vacías.

In [2]:
from colorama import init
init(autoreset=True)

class Color:
    RESET  = "\033[0m"
    GREEN  = "\033[92m"  # Números originales
    BLUE   = "\033[94m"  # Números del usuario
    YELLOW = "\033[93m"  # Pistas
    GRAY   = "\033[90m"  # Casillas vacías

---

## 2. Creación del tablero vacío

Implementamos una función que retorna una matriz 9x9 inicializada en ceros, representando el tablero vacío de Sudoku.

In [3]:
def crear_tablero():
    """
    Devuelve un tablero 9×9 inicializado en 0.
    """
    return [[0 for _ in range(9)] for _ in range(9)]

---

## 3. Verificación de jugadas válidas

Creamos una función que verifica si un número puede colocarse en una posición dada del tablero, cumpliendo las reglas de Sudoku:

- El número no debe estar en la misma fila.
- El número no debe estar en la misma columna.
- El número no debe estar en el mismo subcuadro 3x3.

In [5]:
def es_valido(tablero, fila, col, num):
    # Comprueba fila y columna
    for i in range(9):
        if tablero[fila][i] == num or tablero[i][col] == num:
            return False

    # Determina el inicio del bloque 3×3
    inicio_fila = 3 * (fila // 3)
    inicio_col  = 3 * (col // 3)

    # Comprueba el bloque 3×3
    for i in range(inicio_fila, inicio_fila + 3):
        for j in range(inicio_col, inicio_col + 3):
            if tablero[i][j] == num:
                return False

    return True

---

## 4. Generación de la solución completa (backtracking)

Desarrollamos una función recursiva que llena el tablero usando backtracking, probando números aleatorios en cada celda vacía hasta obtener una solución válida.

In [6]:
import random

def generar_solucion(tablero):
    for fila in range(9):
        for col in range(9):
            if tablero[fila][col] == 0:
                nums = list(range(1, 10))
                random.shuffle(nums)
                for num in nums:
                    if es_valido(tablero, fila, col, num):
                        tablero[fila][col] = num
                        if generar_solucion(tablero):
                            return True
                        tablero[fila][col] = 0
                return False
    return True

---

## 5. Ocultación de celdas para el juego

A partir de la solución completa, eliminamos aleatoriamente una cantidad de celdas para crear el tablero que el usuario debe resolver. Puedes ajustar la cantidad de celdas vacías para cambiar la dificultad.

In [10]:
def generar_juego(tablero, cantidad_vacia=40):
    """
    Devuelve un tablero con 'cantidad_vacia' celdas puestas a 0,
    garantizando que sea humanamente resoluble con pistas.
    """
    juego = [fila[:] for fila in tablero]  # Copia profunda
    vacias = 0
    while vacias < cantidad_vacia:
        fila, col = random.randint(0, 8), random.randint(0, 8)
        if juego[fila][col] != 0:
            juego[fila][col] = 0
            vacias += 1
    return juego

---

## 6. Impresión del tablero con colores

Creamos una función que imprime el tablero en consola, usando los colores definidos para diferenciar números originales, del usuario, pistas y vacíos. También muestra la numeración de filas y columnas para facilitar la interacción.

In [11]:
def imprimir_tablero(tablero, originales):
    # Cabecera de columnas
    print("\n  ", " ".join(str(i+1) for i in range(9)))
    for i, fila in enumerate(tablero):
        print(i+1, end=" ")
        for j, val in enumerate(fila):
            if val == 0:
                # Punto gris para vacío
                print(f"{Color.GRAY}·{Color.RESET}", end=" ")
            elif originales[i][j] == -1:
                # Pista en amarillo
                print(f"{Color.YELLOW}{val}{Color.RESET}", end=" ")
            elif originales[i][j] != 0:
                # Número original en verde
                print(f"{Color.GREEN}{val}{Color.RESET}", end=" ")
            else:
                # Número ingresado en azul
                print(f"{Color.BLUE}{val}{Color.RESET}", end=" ")
        print()

---

## 7. Pistas aleatorias

Desarrollamos una función que revela una celda vacía al azar como pista, marcándola con un color especial y notificando al usuario.

In [12]:
def dar_pista(tablero, solucion, originales):
    opciones = [
        (i, j)
        for i in range(9)
        for j in range(9)
        if tablero[i][j] == 0
    ]
    if not opciones:
        print(f"{Color.YELLOW}No hay más casillas vacías.{Color.RESET}")
        return

    i, j = random.choice(opciones)
    tablero[i][j]    = solucion[i][j]
    originales[i][j] = -1  # Marcamos como pista

    print(
        f"{Color.YELLOW}Pista colocada en fila {i+1}, columna {j+1}.{Color.RESET}"
    )

---

## 8. Bucle principal de juego

Implementamos la función principal que gestiona la interacción con el usuario: inserción de números, solicitud de pistas, impresión del tablero y finalización del juego.

In [ ]:
def jugar():
    # Inicialización
    solucion   = crear_tablero()
    generar_solucion(solucion)

    juego      = generar_juego(solucion)
    originales = [fila[:] for fila in juego]

    # Ciclo de juego
    while True:
        imprimir_tablero(juego, originales)
        accion = input(
            "¿Qué deseas hacer? (insertar [fila col valor], pista, salir): "
        ).strip().lower()

        if accion == "salir":
            print("Gracias por jugar.")
            break

        if accion == "pista":
            dar_pista(juego, solucion, originales)
            continue

        if accion.startswith("insertar"):
            partes = accion.split()
            if len(partes) != 4:
                print("Formato: insertar fila col valor")
                continue

            try:
                fila, col, valor = (
                    int(partes[1]) - 1,
                    int(partes[2]) - 1,
                    int(partes[3]),
                )
            except ValueError:
                print("Fila, columna y valor deben ser números.")
                continue

            # Validaciones
            if (0 <= fila < 9 and 0 <= col < 9 and 1 <= valor <= 9 and
                originales[fila][col] == 0 and
                es_valido(juego, fila, col, valor)):

                juego[fila][col] = valor
            else:
                print(f"{Color.YELLOW}Movimiento inválido o celda bloqueada.{Color.RESET}")
        else:
            print("Comando no reconocido.")

if __name__ == "__main__":
    jugar()